In [ ]:
from __Detection.detection_api import DetectionAPI
from __Tracking.tracking_api import TrackerAPI
from homo_graphy import PlanProjector
import cv2

In [2]:
detector = DetectionAPI()

Loads checkpoint by local backend from path: /workspace/PretrainedModel_by_JeonYT/vehicle/epoch_54.pth
Loads checkpoint by local backend from path: /workspace/PretrainedModel_by_JeonYT/worker/epoch_100.pth


In [3]:
class Args:
    track_thresh = 0.5
    match_thresh = 0.5
    track_buffer = 60
    mot20 = False
args = Args()

In [ ]:
# 1) 도면 + (선택) H 로드
plan_path = "/home/yutak/workspace/MCMT_with_Ensemble/assets/25082_homograph_coordinate-plane.jpg"
plan_pts  = [(861, 1657), #point1
             (861, 1036), #point2
             (861, 412), #point3
             (2225, 1657), #point4
             (2225, 1036), #point5
             (2225, 412), #point6
             (3588, 1657), #point7
             (3588, 1036), #point8
             (3588, 412)    #point9
             ]

In [ ]:
tracker = TrackerAPI(args=args, detector=detector)
cam1_tracklets = tracker.trackingVideo(
    video_path="/workspace/mounted_datasets/Hyundai_dataset/aug1/c1/video.mp4",
    save_path="/workspace/results/tracking_result2.mp4",
    trail_len=100
)

결과 영상 저장 완료: /workspace/results/tracking_result2.mp4


In [ ]:
# 2) 대응점으로 H 추정 (리스트 안에 '튜플' 형태) --> 예시 나중에 바꿔야함
projector = PlanProjector(plan_path, trail_len=60)
cam1_pts = [(100, 200), (400, 210), (395, 600), (120, 590)]
H, mask = projector.fit_homography(cam1_pts, plan_pts, ransac_thresh=3.0)

# 4-1) 예시, 한 프레임만 투영(10번 프레임) & 저장
projected, canvas = projector.projection(cam1_tracklets[10], mode="bottom-center", draw=True)
cv2.imwrite("/workspace/plan_proj_frame0.jpg", canvas)
# 4-2) 전체 프레임 미니맵 비디오 저장
projector.save_video(cam1_tracklets, "/workspace/plan_projection.mp4", fps=30, mode="bottom-center")
